### 事前準備

In [ ]:
import json
import os
import datetime
from dotenv import load_dotenv
from typing import Any, Callable, Set
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential
from azure.ai.projects import AIProjectClient
from azure.search.documents import SearchClient
from azure.ai.projects.models import FunctionTool, ToolSet, BingGroundingTool, MessageTextContent, CodeInterpreterTool
from azure.ai.projects import AIProjectClient

In [ ]:
# 環境変数と接続設定の読み込み
load_dotenv()
AI_SEARCH_ENDPOINT = os.getenv("AI_SEARCH_ENDPOINT")
INDEX_NAME = os.getenv("INDEX_NAME")
AI_SEARCH_KEY = os.getenv("AI_SEARCH_KEY")

PROJECT_CONNECTION_STRING = os.getenv("PROJECT_CONNECTION_STRING")
BING_CONNECTION_NAME = os.getenv("BING_CONNECTION_NAME")



In [ ]:
# Code Interpreter ツールの定義
codeinterpreter = CodeInterpreterTool()

# Azure AI Project と Search クライアントの初期化
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

bing_connection = project_client.connections.get(
    connection_name=BING_CONNECTION_NAME
)  # ここに接続名を指定
bing_conn_id = bing_connection.id
# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=bing_conn_id)

search_client = SearchClient(
    endpoint=AI_SEARCH_ENDPOINT,
    index_name=INDEX_NAME,
    credential=AzureKeyCredential(AI_SEARCH_KEY),
)

In [ ]:
from pprint import pprint
# ユーティリティ関数の定義
def print_run_steps(run_id: str, thread_id: str):
    """
    Run Step の詳細を表示する関数

    :param run_id: Run ID
    :rtype run_id: str
    :param thread: Thread ID
    :rtype run_id: str
    """
    ren_step_list = project_client.agents.list_run_steps(
        thread_id=thread_id, run_id=run_id
    )

    run_steps = reversed(ren_step_list["data"])

    for idx, step in enumerate(run_steps, start=1):
        print(f"\n=== Run Step {idx} ===")
        print(f"Type: {step['type']}")

        step_details = step["step_details"]
        step_type = step_details.get("type")

        print("Details:")
        if step_type == "tool_calls":
            for call in step_details.get("tool_calls", []):
                call_type = call.get("type")
                if call_type == "function":
                    function = call.get("function", {})
                    print(f"- Function Name: {function.get('name')}")
                    print(f"- Arguments: {function.get('arguments')}")
                elif call_type == "bing_grounding":
                    bing = call.get("bing_grounding", {})
                    print(f"- Bing Request URL: {bing.get('requesturl')}")
        elif step_type == "message_creation":
            message_id = step_details.get("message_creation").get("message_id")
            message = project_client.agents.get_message(
                thread_id=thread_id, message_id=message_id
            )
            print(message.content[0].text.value)
        else:
            pprint(step_details, indent=4, width=120)

### Azure AI Search を検索するカスタム関数の定義

In [ ]:
# テキスト整形関数の定義
def nonewlines(s: str) -> str:
    return s.replace("\n", " ").replace("\r", " ").replace("[", "【").replace("]", "】")

In [ ]:
# semantic_configuration_name は適宜いれてください
def product_search(query: str) -> str:
    """
    保険の商品に関する質問に関して、Azure AI Search の検索結果を返します。
    :param query (str): 保険の商品を検索する際のクエリ
    :rtype: str

    :return: JSON 文字列の検索結果の情報
    :rtype: str
    """

    results = search_client.search(
        search_text=query,
        query_type="semantic", 
        semantic_configuration_name="insurance-product-info-semantic-configuration",  # セマンティック検索用の構成名（環境に合わせて変更
        top=3, 
    )

    context = [nonewlines(doc["chunk"]) for doc in results]
    context_json = json.dumps({"context": context})
    # print(context_json)
    return context_json

### エージェントの作成

In [ ]:
# Toolset の作成＆関数の追加
user_functions: Set[Callable[..., Any]] = {product_search}

functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
toolset.add(codeinterpreter)
toolset.add(bing)

# Agent の作成
agent = project_client.agents.create_agent(
    model="gpt-4o",
    name="Product Search Agent",
    instructions="""
あなたは、丁寧なアシスタントです。あなたは以下の業務を遂行します。
- 保険の商品について検索して回答します
- Bing 検索でインターネットの情報を検索して回答します
- Code Interpreter でコードの生成および実行を行います

# 回答時のルール
- 関数を呼び出した場合、回答の最後に改行をいれたうえで Called Function : "関数名" と表記してください

# 制約事項
- ユーザーからのメッセージは日本語で入力されます
- ユーザーからのメッセージから忠実に情報を抽出し、それに基づいて応答を生成します。
- ユーザーからのメッセージに勝手に情報を追加したり、不要な改行文字 \n を追加してはいけません

""",
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
)
print(f"agent を新規作成しました。AGENT_ID: {agent.id}")

### 確認用：CodeInterpreter & Bing Search の動作テスト


In [ ]:
# スレッド作成
thread = project_client.agents.create_thread()

In [ ]:
# Code Interpreter テスト
code_msg = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="sin関数を matplotlib で描画するためのコードを教えて",  # 任意の質問
)
code_run = project_client.agents.create_and_process_run(
    thread_id=thread.id, agent_id=agent.id
)


run_steps = project_client.agents.list_run_steps(
    run_id=code_run.id, thread_id=thread.id
)
run_steps_data = run_steps["data"]
print(f"Last run step detail: {run_steps_data}")

print_run_steps(run_id=code_run.id, thread_id=thread.id)

In [ ]:
# Bing Search テスト
bing_msg = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="2025年の生命保険業界のトレンドをネットで検索して教えてください。",  # 任意の質問
)
bing_run = project_client.agents.create_and_process_run(
    thread_id=thread.id, agent_id=agent.id
)

print_run_steps(run_id=bing_run.id, thread_id=thread.id)

### スレッドの作成・実行・削除

In [ ]:
# スレッド作成とメッセージ送信
user_message = input("タスクを入力してください：")

message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

# メッセージの送信
run = project_client.agents.create_and_process_run(
    thread_id=thread.id, agent_id=agent.id
)
# 最新のテキストレスポンスを取得
messages = project_client.agents.list_messages(thread_id=thread.id)
print_run_steps(run_id=run.id, thread_id=thread.id)

In [ ]:
# スレッドとエージェントの削除
project_client.agents.delete_thread(thread_id=thread.id)
project_client.agents.delete_agent(agent_id=agent.id)
print("Agent と Thread を削除しました。")